Based on the previous task, write a function that will take the name of the file to generate, the buyer's data and the list of items on the invoice - and then generate and save the invoice in a file with the given name.

In the file `invoices data.json` you are going to find data for three invoices – read them in a loop and pass to the function to generate PDF format invoice files. Files should be named `invoice-1.pdf`, `invoice-2.pdf` etc.

In [19]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Image, Spacer, Table, PageBreak, TableStyle
from reportlab.lib.styles import ParagraphStyle
import reportlab.lib.pagesizes as sizes
import reportlab.lib.enums as positions
from reportlab.lib import colors
import json


In [20]:
with open('./invoices data.json') as file:
    data = json.loads(file.read())

In [21]:
keys = data.keys()
keys

dict_keys(['invoices'])

In [22]:
data

{'invoices': [{'date': '1.12.2020',
   'buyer': {'name': 'Company XYZ',
    'address': 'Company XYZ address',
    'nip': 7564984534,
    'regon': 324353453},
   'items': [{'name': 'Rakes', 'price': 4.5, 'amount': 5},
    {'name': 'Watering can', 'price': 15, 'amount': 10},
    {'name': 'Spade', 'price': 6, 'amount': 5}]},
  {'date': '11.11.2020',
   'buyer': {'name': 'Company ABC',
    'address': 'Company ABC address',
    'nip': 8472058276,
    'regon': 437598432},
   'items': [{'name': 'Doll', 'price': 39.9, 'amount': 6},
    {'name': 'Toy car', 'price': 16.5, 'amount': 4},
    {'name': 'Book', 'price': 19.9, 'amount': 10}]},
  {'date': '15.12.2020',
   'buyer': {'name': 'Company IJK',
    'address': 'Company IJK address',
    'nip': 7473092210,
    'regon': 687432543},
   'items': [{'name': 'Rifle', 'price': 25, 'amount': 10},
    {'name': 'BB ammo', 'price': 2.5, 'amount': 100}]}]}

In [23]:
title_style = ParagraphStyle(name="Title Style", fontName="Times-Roman", fontSize=12, alignment=positions.TA_LEFT)
paragraph_style = ParagraphStyle(name="Title Style", fontName="Times-Roman", fontSize=12)
paragraph_style_right = ParagraphStyle(name="Title Style", fontName="Times-Roman", fontSize=12, alignment=positions.TA_RIGHT)

In [24]:
data['invoices'][0]

{'date': '1.12.2020',
 'buyer': {'name': 'Company XYZ',
  'address': 'Company XYZ address',
  'nip': 7564984534,
  'regon': 324353453},
 'items': [{'name': 'Rakes', 'price': 4.5, 'amount': 5},
  {'name': 'Watering can', 'price': 15, 'amount': 10},
  {'name': 'Spade', 'price': 6, 'amount': 5}]}

In [25]:
city = 'Warsaw'
sel_company = 'Company XYZ'
sel_company_adr = 'Company adress'
sel_vat = 3826549374
sel_regon = 709657623

In [26]:
ts = TableStyle([
    ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
    ('FONT', (0, 0), (-1, -1), 'Times-Roman', 12),
    ('ALIGN', (2, 1), (-1, -1), 'RIGHT'),
])

In [27]:
i = 1
for invoice in data['invoices']:
    story = []
    doc = SimpleDocTemplate(f"invoice-{i}.pdf", pagesize=sizes.A4, rightMargin=72, leftMargin=72, topMargin=72, bottomMargin=18)
    story.append(Paragraph(f"Place of issue: {city}", paragraph_style_right))
    story.append(Paragraph(f"Issue date: {invoice['date']}", paragraph_style_right))
    story.append(Paragraph(f"Sale date: {invoice['date']}", paragraph_style_right))
    story.append(Paragraph(f"End date: {invoice['date']}", paragraph_style_right))
    story.append(Spacer(1, 18))

    story.append(Paragraph('Seller:', paragraph_style))
    story.append(Paragraph(sel_company, paragraph_style))
    story.append(Paragraph(sel_company_adr, paragraph_style))
    story.append(Paragraph(f"VAT id (NIP): {sel_vat}", paragraph_style))
    story.append(Paragraph(f"Company registration no. (REGON): {sel_regon}", paragraph_style))
    story.append(Spacer(1, 18))

    story.append(Paragraph('Buyer:', paragraph_style))
    story.append(Paragraph(f"{invoice['buyer']['name']}", paragraph_style))
    story.append(Paragraph(f"{invoice['buyer']['address']}", paragraph_style))
    story.append(Paragraph(f"VAT id (NIP): {invoice['buyer']['nip']}", paragraph_style))
    story.append(Paragraph(f"Company registration no. (REGON): {invoice['buyer']['regon']}", paragraph_style))
    story.append(Spacer(1, 30))

    items = []
    x = 1
    headers = ['No.', 'Item', 'Price', 'Amount', 'Total Price']
    items.append(headers)
    for item in invoice['items']:
        item_values = [x, item['name'], item['price'], item['amount']]
        item_total_price = item['price'] * item['amount']
        item_values.append(item_total_price)
        items.append(item_values)
        x += 1 
    table_summary_total = sum(item[4] for item in items[1:])
    table_summary = ['', '', '', 'Total:', table_summary_total]
    items.append(table_summary)
    story.append(Table(data = items, style = ts))
    story.append(Spacer(1, 80))

    story.append(Paragraph("................................", paragraph_style))
    story.append(Paragraph("Signature of the authorized recipient", paragraph_style))
    story.append(Spacer(1, 80))

    story.append(Paragraph("................................", paragraph_style))
    story.append(Paragraph("Signature of the authorized issuer", paragraph_style))
    doc.build(story)
    i = i + 1